In [26]:
import pandas as pd
df = pd.read_csv('full_resample.csv')

In [27]:
import warnings
warnings.filterwarnings("ignore")

Decided to do additional modelling with boosting for the hitting stats.  This will be done with gridsearch

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import AdaBoostClassifier

Dropping the columns made without resampling

In [29]:
df.drop(columns=['traditional predictions', 'sabr_predictions'], inplace=True)

In [30]:
no_fun = ['H', 'RBI', 'HR', 'SO', 'AVG', 'OBP', 'OPS']
sabr = ['R', 'BB', 'IBB', 'OPS+', 'wRC+', 'WAR', 'TTO%']

In [31]:
df[no_fun].head()

,H,RBI,HR,SO,AVG,OBP,OPS
0,143,111,42,109,0.285,0.357,0.943
1,188,80,21,81,0.303,0.336,0.806
2,177,120,32,109,0.301,0.379,0.908
3,127,68,26,102,0.251,0.340,0.807
4,147,55,12,97,0.263,0.342,0.739


In [32]:
df[sabr].head()

,R,BB,IBB,OPS+,wRC+,WAR,TTO%
0,110,56,2,120,136,4.7,0.362
1,88,34,8,90,98,2.4,0.206
2,88,74,10,115,129,7.3,0.320
3,75,61,1,98,109,1.6,0.325
4,102,63,0,78,94,2.3,0.272


In [33]:
params = {
    'penalty': ['l1', 'l2'],
    'C': [1, 2,3,4,5,6,7,8,9,10]
}

In [34]:
grid = GridSearchCV(LogisticRegression(), param_grid=params)

In [35]:
X_t = df[no_fun]
X_s = df[sabr]
y = df['MVP']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_t, y, stratify = y)

In [37]:
grid.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [38]:
a = grid.best_params_

In [39]:
ada_t = AdaBoostClassifier(base_estimator=LogisticRegression(C=a['C'], penalty=a['penalty']))

In [40]:
ada_t.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          learning_rate=1.0, n_estimators=50, random_state=None)

In [41]:
preds = ada_t.predict(df[no_fun])

In [42]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score

In [43]:
def metric_maker(y1, y2):
    
    '''Prints out a bunch of metrics and generates a confusion matrix'''
    
    print(f' Accuracy is {accuracy_score(y1, y2)}')
    
    print(f' Recall is {recall_score(y1,y2)}')
    
    print(f' Specificity is {recall_score(y1, y2, pos_label=0)}')

    return confusion_matrix(y1, y2)

In [44]:
metric_maker(y, preds)

 Accuracy is 0.9293333333333333
 Recall is 0.49333333333333335
 Specificity is 0.9777777777777777


array([[2640,   60],
       [ 152,  148]])

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X_s, y, stratify = y)

In [47]:
grid.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [48]:
a = grid.best_params_

In [49]:
ada_s = AdaBoostClassifier(LogisticRegression(C=a['C'], penalty = a['penalty']))

In [50]:
ada_s.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          learning_rate=1.0, n_estimators=50, random_state=None)

In [51]:
preds2= ada_s.predict(df[sabr])

In [52]:
metric_maker(y, preds2)

 Accuracy is 0.9266666666666666
 Recall is 0.47333333333333333
 Specificity is 0.977037037037037


array([[2638,   62],
       [ 158,  142]])